In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import os
import glob
import pickle
import random
import matplotlib.pyplot as plt
import numpy as np
import tqdm
from time import time
# %matplotlib inline


from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split


import torch
import torchvision
import torch.nn as nn
from torch.utils import data
from torchvision import transforms,datasets
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable

torch.manual_seed(42)
random.seed(42)
np.random.seed(42)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
def read_data(my_train_transform:transforms,my_train_transform2:transforms,my_test_transform:transforms,batch_size):
  train_dataset=datasets.ImageFolder(root="/content/drive/MyDrive/Data/Train",transform=my_train_transform)
  test_dataset=datasets.ImageFolder(root="/content/drive/MyDrive/Data/Test",transform=my_test_transform)
  test_dataset2=datasets.ImageFolder(root="/content/drive/MyDrive/Data/Test",transform=my_test_transform)
  train_dataset2=datasets.ImageFolder(root="/content/drive/MyDrive/Data/Train",transform=my_train_transform2)
  new_trained=torch.utils.data.ConcatDataset([train_dataset,train_dataset2])
  new_test_dataset=torch.utils.data.ConcatDataset([test_dataset,test_dataset2])
  print(len(new_trained))
  print(type(new_trained))
  train_data_loader=DataLoader(new_trained,batch_size=batch_size,shuffle=True)
  test_data_loader=DataLoader(test_dataset,batch_size=batch_size,shuffle=False)
  classes=train_dataset.classes
  print(classes)
  return train_data_loader,test_data_loader,classes

In [5]:
my_test_transform=transforms.Compose([torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize((227,227)),
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.3, 0.3, 0.3])])


my_train_transform=transforms.Compose([torchvision.transforms.ToTensor(),
                                torchvision.transforms.Resize((227,227)),
                                torchvision.transforms.RandomHorizontalFlip(),
                                torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.3, 0.3, 0.3])])

my_train_transform2=transforms.Compose([torchvision.transforms.ToTensor(),
        torchvision.transforms.Resize((227,227)),
        torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.3, 0.3, 0.3])])


train_data_loader,test_data_loader,class_names=read_data(my_train_transform,my_train_transform2,my_test_transform,batch_size=200)

5970
<class 'torch.utils.data.dataset.ConcatDataset'>
['Bedroom', 'Coast', 'Forest', 'Highway', 'Industrial', 'Inside_City', 'Kitchen', 'Livingroom', 'Mountain', 'Office', 'Open_Country', 'Store', 'Street', 'Suburb', 'Tall_Building']


In [6]:
from torch.nn.modules.dropout import Dropout
from torch.nn.modules.pooling import MaxPool1d
class AlexNet1(nn.Module):

  def __init__(self,output_size:int):
    super(AlexNet1, self).__init__()
    self.conv_layers=nn.Sequential(                         # B 3 227 227
        nn.Conv2d(3,96,kernel_size=11,stride=4,padding=2),  # B 96 56 56
        nn.ReLU(inplace=True),                              # B 96 56 56
        nn.MaxPool2d(kernel_size=4,stride=4),               # B 96 14 14
        nn.Dropout2d(p=0.5),
        nn.BatchNorm2d(96)
      )
    
    self.fullyConnected=nn.Sequential(
        nn.Flatten(),
        nn.Dropout(p=0.5),
        nn.Linear(96*14*14,4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096,output_size)
     )
    

  def forward(self,x:torch.tensor) -> torch.tensor:
    x=self.conv_layers(x)
    x=self.fullyConnected(x)
    return x

In [7]:
class AlexNet2(nn.Module):

  def __init__(self,output_size:int):
    super(AlexNet2, self).__init__()
    self.conv_layers=nn.Sequential(                         # B 3 227 227
        nn.Conv2d(3,96,kernel_size=11,stride=4,padding=2),  # B 96 56 56
        nn.ReLU(inplace=True),                              # B 96 56 56
        nn.MaxPool2d(kernel_size=4,stride=4),               # B 96 14 14
        nn.Dropout2d(p=0.5),                                # B 96 14 14
        nn.BatchNorm2d(96),                                 # B 96 14 14
        nn.Conv2d(96,256,kernel_size=5,stride=1,padding=2),  # B 256 14 14
        nn.ReLU(inplace=True),                              # B 256 14 14
        nn.MaxPool2d(kernel_size=3,stride=2),               # B 256 6  6
        nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1), # B 256 6 6
        nn.ReLU(inplace=True),                              # B 256 6 6
        nn.MaxPool2d(kernel_size=2,stride=2),                # B 256 3 3
        nn.Dropout2d(p=0.5),                               
        nn.BatchNorm2d(256)
    )
    
    self.fullyConnected=nn.Sequential(
        nn.Flatten(),
        nn.Dropout(p=0.5),
        nn.Linear(256*3*3,4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096,4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096,output_size)
     )
    

  def forward(self,x:torch.tensor) -> torch.tensor:
    x=self.conv_layers(x)
    x=self.fullyConnected(x)
    return x

In [8]:
class AlexNet3(nn.Module):

  def __init__(self,output_size:int):
    super(AlexNet3, self).__init__()
    self.conv_layers=nn.Sequential(                         # B 3 227 227
        nn.Conv2d(3,96,kernel_size=11,stride=4,padding=0),  # B 96 55 55
        nn.ReLU(inplace=True),                              # B 96 55 55
        nn.MaxPool2d(kernel_size=3,stride=2),               # B 96 27 27
        # nn.Dropout2d(p=0.5),                               
        # nn.BatchNorm2d(96),
        nn.Conv2d(96,256,kernel_size=5,stride=1,padding=2),  # B 256 27 27
        nn.ReLU(inplace=True),                              # B 256 27 27
        nn.MaxPool2d(kernel_size=3,stride=2),               # B 256 13 13
        # nn.Dropout2d(p=0.5),                               
        # nn.BatchNorm2d(256),
        nn.Conv2d(256,384,kernel_size=3,stride=1,padding=1), # B 384 13 13
        nn.ReLU(inplace=True),                              # B 384 13 13
        nn.Conv2d(384,384,kernel_size=3,stride=1,padding=1),# B 384 13 13
        nn.ReLU(inplace=True),                              # B 384 13 13
        nn.Conv2d(384,256,kernel_size=3,stride=1,padding=1),# B 256 13 13
        nn.ReLU(inplace=True),                              # B 256 13 13
        nn.MaxPool2d(kernel_size=3,stride=2),               # B 256 6 6
        nn.Dropout(p=0.5)                                   # B 256 6 6
    )
    
    self.fullyConnected=nn.Sequential(
        nn.Flatten(),
        nn.Dropout(p=0.5),
        nn.Linear(256*6*6,4096),
        nn.ReLU(inplace=True),
        nn.Dropout(p=0.5),
        nn.Linear(4096,4096),
        nn.ReLU(inplace=True),
        nn.Linear(4096,output_size)
     )
    

  def forward(self,x:torch.tensor) -> torch.tensor:
    x=self.conv_layers(x)
    x=self.fullyConnected(x)
    return x

In [9]:
alexnet1=AlexNet1(len(class_names))
alexnet2=AlexNet2(len(class_names))
alexnet3=AlexNet3(len(class_names))
alexnet1=alexnet1.to(device)
alexnet2=alexnet2.to(device)
alexnet3=alexnet3.to(device)

In [10]:
fine_tunning_model=torchvision.models.alexnet(pretrained=True)
numOfLastLayerFeatures=fine_tunning_model.classifier[-1].in_features
fine_tunning_model.classifier[-1]=nn.Linear(numOfLastLayerFeatures,15)
fine_tunning_model=fine_tunning_model.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

In [11]:
transferLearningModel=torchvision.models.alexnet(pretrained=True)
for param in transferLearningModel.parameters():
  param.requires_grad=False

numOfLastLayerFeatures=transferLearningModel.classifier[-1].in_features
transferLearningModel.classifier[-1]=nn.Linear(numOfLastLayerFeatures,15)
transferLearningModel=transferLearningModel.to(device)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
def train_epoch(model:nn.Module,learnin_rate,momentum):
  global train_data_loader
  criterion=nn.CrossEntropyLoss().to(device)
  optimizer=torch.optim.SGD(model.parameters(),lr=learnin_rate,momentum=momentum)

  epoch_loss=0
  epoch_accuracy=0
  epoch_accuracy5=0
  epoch_true=0
  epoch_true5=0
  all=0

  for i ,(images,labels) in enumerate(train_data_loader):
    print(i)
    images=images.to(device)
    labels=labels.to(device)
    model.train()

    y=model(images)

    loss=criterion(y,labels)
  
    epoch_loss= epoch_loss+float(loss.item())

    prediction=y.argmax(-1)
    _,prediction5=torch.topk(y,k=5)

    all=all+len(prediction)
    a=(prediction==labels).sum()
    epoch_true=epoch_true+a
    for j in range(5):
      epoch_true5=epoch_true5+(prediction5[:,j]==labels).sum().item()
    

    loss.backward()
    optimizer.step()

    optimizer.zero_grad()

  epoch_loss=epoch_loss/float(len(train_data_loader))
  epoch_accuracy=epoch_true/float(all)
  epoch_accuracy5=epoch_true5/float(all)
  return epoch_accuracy,epoch_accuracy5,epoch_loss

In [13]:
def eval_epoch(model:nn.Module,learnin_rate,momentum):
  global test_data_loader
  criterion=nn.CrossEntropyLoss().to(device)
  optimizer=torch.optim.SGD(model.parameters(),lr=learnin_rate,momentum=momentum)

  epoch_loss=0
  epoch_accuracy=0
  epoch_accuracy5=0
  epoch_true=0
  epoch_true5=0
  all=0
  with torch.no_grad():
    for i ,(images,labels) in enumerate(test_data_loader):
      # print(i)
      images=images.to(device)
      labels=labels.to(device)

      model.eval()

      y=model(images)

      loss=criterion(y,labels)

      epoch_loss= epoch_loss+float(loss.item())

      prediction=y.argmax(-1)
      _,prediction5=torch.topk(y,k=5)
      all=all+len(prediction)
      a=(prediction==labels).sum().item()
      epoch_true=epoch_true+a
      for j in range(5):
        epoch_true5=epoch_true5+(prediction5[:,j]==labels).sum().item()
    
  epoch_loss=epoch_loss/float(len(test_data_loader))
  epoch_accuracy=epoch_true/float(all)
  epoch_accuracy5=epoch_true5/float(all)
  return epoch_accuracy,epoch_accuracy5,epoch_loss

In [14]:
def epoch_for(model:nn.Module,num_of_epoch,learning_rate,momentum,network_name):
  train_loss_list=[]
  test_loss_list=[]
  accuracy_list=[]
  accuracy5_list=[]
  epoch_list=[]
  for epoch_index in range(num_of_epoch):
    current_time=time()
    train_epoch_accuracy,train_epoch_accuracy5,train_epoch_loss=train_epoch(model,learning_rate,momentum)
    print("epoch for :")
    test_epoch_accuracy,test_epoch_accuracy5,test_epoch_loss=eval_epoch(model,learning_rate,momentum)
    print("epoch for2 :")
    epoch_list.append(epoch_index)
    train_loss_list.append(train_epoch_loss)
    test_loss_list.append(test_epoch_loss)
    accuracy_list.append(test_epoch_accuracy)
    accuracy5_list.append(test_epoch_accuracy5)
    duration=time()-current_time
    darw_plots(train_loss_list,test_loss_list,accuracy_list,accuracy5_list,epoch_list,network_name)
    print(f"Epoch: {epoch_index}, Train: Loss: {train_epoch_loss}, Accuracy: {train_epoch_accuracy}, Accuracy5: {train_epoch_accuracy5}")
    print(f"Test: Loss: {test_epoch_loss}, Accuracy: {test_epoch_accuracy}, Accuracy5: {test_epoch_accuracy5}, time={duration}")

  # print(accuracy_list)
  return train_loss_list,test_loss_list,accuracy_list,accuracy5_list,epoch_list


In [15]:
def darw_plots(train_loss_list,test_loss_list,accuracy_list,accuracy5_list,epoch_list,network_name):
  # print(type(accuracy_list))
  # print(type(accuracy5_list))
  # print(type(epoch_list))
  # print(type(train_loss_list))
  # print(type(test_loss_list))
  plt.plot(epoch_list,train_loss_list,label="train")
  plt.plot(epoch_list,test_loss_list,label="test")
  plt.xlabel("epoch")
  plt.ylabel("loss")
  plt.legend()
  plt.savefig(f"/content/drive/MyDrive/{network_name}_Loss")
  plt.clf()


  plt.plot(epoch_list, accuracy_list)
  plt.xlabel('epoch')
  plt.ylabel('accuracy')
  plt.savefig(f"/content/drive/MyDrive/{network_name}_Accuracy")
  plt.clf()

  plt.plot(epoch_list, accuracy5_list)
  plt.xlabel('epoch')
  plt.ylabel('accuracy5')
  plt.savefig(f"/content/drive/MyDrive/{network_name}_Accuracy5")
  plt.clf()



In [16]:
def train_eval_model(model:nn.Module,learnin_rate,momentum,num_of_epochs,model_name):
  train_loss_list,test_loss_list,accuracy_list,accuracy5_list,epoch_list=epoch_for(model,num_of_epochs,learnin_rate,momentum,model_name)
  darw_plots(train_loss_list,test_loss_list,accuracy_list,accuracy5_list,epoch_list,model_name)

In [ ]:
# train_eval_model(alexnet1,0.01,momentum=0,num_of_epochs=100,model_name="res01")

In [ ]:
train_eval_model(alexnet2,0.01,0,100,"res02")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 0, Train: Loss: 2.670075043042501, Accuracy: 0.09530988335609436, Accuracy5: 0.43484087102177554
Test: Loss: 2.6879845559597015, Accuracy: 0.10466666666666667, Accuracy5: 0.4106666666666667, time=31.356545448303223
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 1, Train: Loss: 2.636851660410563, Accuracy: 0.12395309656858444, Accuracy5: 0.4750418760469012
Test: Loss: 2.6564473509788513, Accuracy: 0.13066666666666665, Accuracy5: 0.4826666666666667, time=31.205613374710083
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 2, Train: Loss: 2.5885204633076984, Accuracy: 0.14874371886253357, Accuracy5: 0.5368509212730318
Test: Loss: 2.6202181577682495, Accuracy: 0.13533333333333333, Accuracy5: 0.49933333333333335, time=31.86454129219055
0
1
2
3
4
5
6
7
8
9


<Figure size 432x288 with 0 Axes>

In [ ]:
train_eval_model(alexnet3,0.01,0,100,"res03")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 0, Train: Loss: 2.706418212254842, Accuracy: 0.08542713522911072, Accuracy5: 0.37554438860971523
Test: Loss: 2.7083482146263123, Accuracy: 0.08266666666666667, Accuracy5: 0.3333333333333333, time=31.991924047470093
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 1, Train: Loss: 2.703570604324341, Accuracy: 0.0974874347448349, Accuracy5: 0.41758793969849245
Test: Loss: 2.7078261971473694, Accuracy: 0.06733333333333333, Accuracy5: 0.344, time=32.75182271003723
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 2, Train: Loss: 2.7007858991622924, Accuracy: 0.1072026789188385, Accuracy5: 0.43266331658291457
Test: Loss: 2.707351863384247, Accuracy: 0.06666666666666667, Accuracy5: 0.3333333333333333, time=32.054423570632935
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


<Figure size 432x288 with 0 Axes>

In [17]:
train_eval_model(transferLearningModel,0.01,0,100,"transferLearningModel")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 0, Train: Loss: 1.069558988014857, Accuracy: 0.6978224515914917, Accuracy5: 0.9346733668341709
Test: Loss: 0.6421740911900997, Accuracy: 0.8053333333333333, Accuracy5: 0.9926666666666667, time=3248.3946182727814
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 1, Train: Loss: 0.5398133824268977, Accuracy: 0.8395310044288635, Accuracy5: 0.9943048576214405
Test: Loss: 0.5173844918608665, Accuracy: 0.8386666666666667, Accuracy5: 0.9926666666666667, time=30.225863933563232
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
epoch for :
epoch for2 :
Epoch: 2, Train: Loss: 0.452954426407814, Accuracy: 0.8604689836502075, Accuracy5: 0.9959798994974874
Test: Loss: 0.4645622279495001, Accuracy: 0.854, Accuracy5: 0.9953333333333333, time=30.099436044692993
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16

<Figure size 432x288 with 0 Axes>

In [ ]:
train_eval_model(fine_tunning_model,0.01,0,100,"fine_tunning_model")